This is notebook is about the first assignement from the udacity deep learning course the assigment is about  MNIST handwritten recogniton

first of all let import the usefull libraries

In [12]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle
import imageio
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline


First, we'll download the dataset to our local machine. The data consists of characters rendered in a variety of fonts on a 28x28 image. The labels are limited to 'A' through 'J' (10 classes). The training set has about 500k and the testset 19000 labeled examples. Given these sizes, it should be possible to train models quickly on any machine.

In [2]:
url = 'https://commondatastorage.googleapis.com/books1000/'
last_percent_reported = None
data_root = '../datasets' 

def download_progress_hook(count, blockSize, totalSize):
  """
  A hook to report the progress of a download. This is mostly intended for users with
  slow internet connections. Reports every 5% change in download progress.
  
  """
  global last_percent_reported
  percent = int(count * blockSize * 100 / totalSize)

  if last_percent_reported != percent:
    if percent % 5 == 0:
      sys.stdout.write("%s%%" % percent)
      sys.stdout.flush()
    else:
      sys.stdout.write(".")
      sys.stdout.flush()
      
    last_percent_reported = percent
    
def maybe_download(filename, expected_bytes, force=False):
  """Download a file if not present, and make sure it's the right size."""
  dest_filename = os.path.join(data_root, filename)
  if force or not os.path.exists(dest_filename):
    print('Attempting to download:', filename) 
    filename, _ = urlretrieve(url + filename, dest_filename, reporthook=download_progress_hook)
    print('\nDownload Complete!')
  statinfo = os.stat(dest_filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', dest_filename)
  else:
    raise Exception(
      'Failed to verify ' + dest_filename + '. Can you get to it with a browser?')
  return dest_filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)
        

Attempting to download: notMNIST_large.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified ../datasets/notMNIST_large.tar.gz
Attempting to download: notMNIST_small.tar.gz
0%....5%....10%....15%....20%....25%....30%....35%....40%....45%....50%....55%....60%....65%....70%....75%....80%....85%....90%....95%....100%
Download Complete!
Found and verified ../datasets/notMNIST_small.tar.gz


Once the file downloaded We need to extract the dataset : 
    here is the code for that 

In [3]:
num_classes = 10
np.random.seed(133)

def maybe_extract(filename, force=False):
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  if os.path.isdir(root) and not force:
    # You may override by setting force=True.
    print('{} already present - Skipping extraction of {}.'.format(root, filename))
  else:
    print('Extracting data for {} . This may take a while. Please wait.'.format(root))
    tar = tarfile.open(filename)
    sys.stdout.flush()
    tar.extractall(data_root)
    tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root))
    if os.path.isdir(os.path.join(root, d))]
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders
  
train_folders = maybe_extract(train_filename)
test_folders = maybe_extract(test_filename)

Extracting data for ../datasets/notMNIST_large . This may take a while. Please wait.
['../datasets/notMNIST_large/A', '../datasets/notMNIST_large/B', '../datasets/notMNIST_large/C', '../datasets/notMNIST_large/D', '../datasets/notMNIST_large/E', '../datasets/notMNIST_large/F', '../datasets/notMNIST_large/G', '../datasets/notMNIST_large/H', '../datasets/notMNIST_large/I', '../datasets/notMNIST_large/J']
Extracting data for ../datasets/notMNIST_small . This may take a while. Please wait.
['../datasets/notMNIST_small/A', '../datasets/notMNIST_small/B', '../datasets/notMNIST_small/C', '../datasets/notMNIST_small/D', '../datasets/notMNIST_small/E', '../datasets/notMNIST_small/F', '../datasets/notMNIST_small/G', '../datasets/notMNIST_small/H', '../datasets/notMNIST_small/I', '../datasets/notMNIST_small/J']


In [2]:
from IPython.display import Image

In [5]:
sample_image = Image(filename='../datasets/notMNIST_large/A/em9ya29uLnR0Zg==.png')

In [6]:
sample_image

Let Now load the images in a managable format the images will be loaded in a 3D array
the shape of the array will be (number of training example , 28, 28)

In [20]:
image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

In [24]:
def load_letter(folder, min_num_images):
    
    
    """
    Load the data for a single letter label.
    """
    image_files = os.listdir(folder) #list all the files in the directory of an given image
    dataset = np.ndarray(shape=(len(image_files), image_size, image_size),
                         dtype=np.float32)
    print(folder)
    num_images = 0
    for image in image_files:
        image_file = os.path.join(folder, image)
        try:
            #reading the image and convert it to a matrix of each pixel and normalize them 
            image_data = (imageio.imread(image_file).astype(float) - pixel_depth / 2) / pixel_depth
            if image_data.shape != (image_size, image_size):
                raise Exception('Unexpected image shape: {}'.format(str(image_data.shape)))
            dataset[num_images, :, :] = image_data #adding the image to the dataset
            num_images +=1
        except (IOError, ValueError) as e:
            print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    dataset = dataset[0:num_images, :, :]
    if num_images < min_num_images:
        raise Exception('Many fewer images than expected: {} < {}'.format(num_images, min_num_images))
    print('Full dataset tensor:', dataset.shape)
    print('Mean:', np.mean(dataset))
    print('Standard deviation:', np.std(dataset))
    return dataset
            

In [25]:
dataset_a = load_letter('../datasets/notMNIST_large/A/',1000)

../datasets/notMNIST_large/A/
Could not read: ../datasets/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Could not read: ../datasets/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Could not read: ../datasets/notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : Could not find a format to read the specified file in mode 'i' - it's ok, skipping.
Full dataset tensor: (52909, 28, 28)
Mean: -0.12825
Standard deviation: 0.443121


In [27]:
def maybe_pickle(data_folders, min_num_images_per_class, force=False):
    """
    pickle some some training example from images dataset
    """
    dataset_names = []
    for folder in data_folders:
        set_filename = folder + '.pickle'
        dataset_names.append(set_filename)
    if os.path.exists(set_filename) and not force:
        # You may override by setting force=True.
        print('{} already present - Skipping pickling.'.format(set_filename))
    else:
        print('Pickling {}.'.format(set_filename))
        dataset = load_letter(folder, min_num_images_per_class)
        try:
            with open(set_filename, 'wb') as f:
                pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)
        except Exception as e:
            print('Unable to save data to', set_filename, ':', e)
    return dataset_names


In [ ]:
tarinfloa

In [28]:
train_datasets = maybe_pickle(train_folders, 45000)
test_datasets = maybe_pickle(test_folders, 1800)

NameError: name 'train_folders' is not defined